In [1]:
import os
os.environ['NUMBA_WARNINGS'] = '1'
import numpy as np
import numba

import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt

from matrixmethod import reflec_and_trans
import matrixmethod_numba
from matrixmethod_numba import reflec_and_trans as reflec_and_trans_numba, fields

/home/mpfluege/.local/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
from Anton_XRR import reflectivity
def anton_xrr(ang_deg, wl, n, rough):
    l = len(ang_deg)
    AOI = 90.-ang_deg
    wavelengths = np.array([wl]*l)
    n = np.outer(n, np.ones(l))
    t = np.concatenate([thick, [0]])
    t = np.outer(t, np.ones(l))
    sigma = rough
    kz_AOI, kx_AOI = reflectivity.helper.k_z_generator(AOI, wavelengths, n)
    r, trans = reflectivity.matrixmethod_xrr.amplitudes(n, wavelengths, kz_AOI, t, sigma)
    return r, trans

In [3]:
n = np.array([1] + [1-1e-5+1e-6j]*20 + [1-2e-5+2e-6j] + [1])
rough = np.array([0.5]*22)
thick = np.array([1.]*21)
wl = 0.15
ang_deg = np.linspace(0, 1, 101)[1:]
ang = np.deg2rad(ang_deg)

In [4]:
#%timeit reflec_and_trans(n, wl, ang, thick, rough)
%timeit reflec_and_trans_numba(n, wl, ang, thick, rough)
%timeit fields(n, wl, ang, thick, rough)
%timeit anton_xrr(ang_deg, wl, n, rough)

The slowest run took 966.13 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 529 µs per loop


LoweringError: Failed at nopython (nopython mode backend)
Internal error:
NotImplementedError: list(array(complex128, 1d, C)): unsupported nested memory-managed object
File "matrixmethod_numba.py", line 220

In [ ]:
mr, mt = reflec_and_trans(n, wl, ang, thick, rough)
nr, nt = fields(n, wl, ang, thick, rough)
nr = np.array(nr)
nt = np.array(nt)
ar, at = anton_xrr(ang_deg, wl, n, rough)

In [ ]:
plt.figure()
plt.plot(ang_deg, np.abs(mr)**2, label='simple r')
#plt.plot(ang_deg, np.abs(np.array(Ap).T[0])**2, label='numba A+')
plt.plot(ang_deg, np.abs(ar[0])**2, label='anton r')
plt.plot(ang_deg, np.abs(nr[:,0])**2, label='numba r')
plt.legend(loc='lower right')
plt.yscale('log')
plt.figure()
#plt.plot(ang_deg, np.abs(np.array(Am).T[-1])**2, label='numba A-')
plt.plot(ang_deg, np.abs(at[-1])**2, label='anton t')
plt.plot(ang_deg, np.abs(mt)**2, label='simple t')
plt.plot(ang_deg, np.abs(nt[:,0])**2, label='numba t')
plt.yscale('log')
plt.legend(loc='lower right')

In [ ]:
nr[:,-1]

In [ ]:
ar.shape

In [ ]:
Ams = np.array(Am).T

In [ ]:
nt

In [ ]:
Aps = np.array(Ap).T
Aps.shape
Ams = np.array(Am).T
Ams.shape

In [ ]:
(rr - rrr[0]).max()

In [ ]:
rr

In [ ]:
np.abs(np.array(tt))**2

In [ ]:
np.abs(np.array(t))**2